In [ ]:
import pandas as pd
import numpy as np
import random

In [2]:
with open("./data/causal_inference.txt", 'r') as f:
    text = f.readlines()

In [3]:
res = []
for t in text:
    res.append(list(map(int, t.strip('\n').split(" ")[:-1]))) # 最後空文字が入っている。
#     print(t.strip('\n').split(" ")[:-1])

# コロンビア大学で用いられているサンプルデータセット
http://www.cs.columbia.edu/~jebara/6998/hw2.pdf

- the 1st column is the chosen arm
- the 2nd column is binary reward 
- the remaining columns are the context features.

In [4]:
ds = pd.DataFrame(res)
ds.columns = ["arm", "reward"] + [f"feat_{i}" for i in range(100)]

In [5]:
X = ds.loc[:, "feat_0":].values
y = ds.loc[:, "reward"].values
arms = ds.loc[:,"arm"].values

In [6]:
import os, sys

sys.path.append(".")

In [7]:
from src.Arms import ContextualArms

In [8]:
import itertools, os
ds.groupby("arm").mean()["reward"].idxmax()

2

In [9]:
Arms = ContextualArms(n_action=10, max_n_sim=100, X=X, y=y,arms=arms, n_features=X.shape[1])

In [10]:
Arms

In [11]:
d = 100
sigma=1; sigma_0=1
A_inv =sigma_0/sigma * np.eye(d); A_inv
b_t = np.zeros((d, 1))
alpha = 0.05
n_arms = 10
n_sim = 10
history = dict(chosen_arms=[],rewards=[])
mu_hat = np.zeros((n_arms, 1))

In [12]:
#  Thompson sampling アルゴリズム
for t in range(1, n_sim):
    # 3. draw_theta
    theta = np.random.multivariate_normal(A_inv@b_t.flatten(), sigma**2*A_inv, size=1)
    # 4. A[t]はa_itの転置ver
    # 各アームの特徴量を引く
    # shape : (10, 1, 100) 10 arms, (1, 100)の特徴量ベクトル
    # 1次元目にアーム１から10までの特徴量を順番に格納している。
    A_ = Arms.draw()
    for i in range(n_arms):
        mu_hat[i] = np.dot(A_[i].ravel(), theta.ravel())
    i_star = mu_hat.argmax(0)[0]
    history["chosen_arms"].append(i_star)

    #5. スコア最大の行動 i*　を選択して報酬を観測する。
    reward = Arms.get_reward(index=i_star)
    history["rewards"].append(reward)
    #6.  A-inverse/ bの更新をする。
    second_term = (A_inv@A_[i_star].T@A_[i_star]@A_inv) / (1+A_[i_star]@A_inv@A_[i_star].T)
    A_inv = A_inv - second_term
    b_t += A_[i_star].T*reward    

# Logit Regression

- logit モデルは対数事後確率の勾配ベクトルから陽に（解析的に）$\theta$を求めることが難しい。

$- \log\pi (\theta| \{i(s), X(s)\}^t_{s=1})$
$\displaystyle = \frac{\theta^T \theta}{2\sigma^2_0} + \sum^T_{s=1}log(1+\exp(\theta^Ta_{i(s), s})) - \sum^T_{s:X(s)=1}\theta^Ta_{i(s), s} + Constants. $


$G_t(\theta) = - \nabla\log\pi (\theta| \{i(s), X(s)\}^t_{s=1})$
$\displaystyle = \frac{\theta}{2\sigma^2_0} + \sum^t_{s=1}\frac{e^{\theta^Ta_{i(s), s}}a_{i(s), s}}{(1+\exp(\theta^Ta_{i(s), s}))} - \sum^T_{s:X(s)=1}a_{i(s), s}$

- $G_t(\theta) = 0が\theta$について解けない


- そこで、数値計算をする。ニュートン法を用いて、負の対数事後確率を$\hat{\theta}_{MAP}$の周りで２次近似すると、($G_t(\hat{\theta_{MAP}}) = 0$より)

- 負の対数事後確率が多変量正規分布の負の対数ゆうどに近似される。これをLaplas近似というらしい。
    - $\theta_{MAP} \sim N(\hat{\theta_{MAP}}, H_t(\hat{\theta_{MAP}})^{-1})$

In [13]:
def component_logit_hessian(theta, arm_feature):
    return np.exp(np.dot(theta.ravel(), arm_feature.ravel()))* arm_feature.T@arm_feature /(1+np.exp(np.dot(theta.ravel(), arm_feature.ravel())))**2
def Hessian_inv(theta, Arms=None):
    assert Arms is not None
    hessian = np.eye(d)/sigma_0
    # self.cum_matrix
    # initial: self.cum_matrix = np.zeros((d, d))
    for i in np.array(Arms.history_idx).flatten():
        x, y = Arms[i]
        hessian = hessian + component_logit_hessian(theta, x.reshape((1, -1)))
    return np.linalg.inv(hessian)

def Gradient(theta, sigma_0,  Arms):
    '''gradients vector for logit based posterior probability.'''
    first_term = theta/sigma_0
    second_term = 
    

In [14]:
# ロジスティックモデル上のトンプソン抽出
# WRITE CODE
# Newton method

# 6. draw_theta
theta = np.random.multivariate_normal(A_inv@b_t.flatten(), sigma**2*A_inv, size=1)
# 各アームの特徴量を引く
# shape : (10, 1, 100) 10 arms, (1, 100)の特徴量ベクトル
# 1次元目にアーム１から10までの特徴量を順番に格納している。
A_ = Arms.draw()
for i in range(n_arms):
    mu_hat[i] = np.dot(A_[i].ravel(), theta.ravel())
i_star = mu_hat.argmax(0)[0]
history.append(i_star)



In [16]:
def newton_iteration(theta, Arms):
    assert Arms is not None
Arms.history_idx

[[4183], [7861], [1677], [3619], [3889], [7830], [9463], [9184], [8481]]

# Calculate Regrets.

In [ ]:
# 関数にするかも
class RegretCalculator():
    '"EnvとAgentのヒストリーを踏まえて、期待リグレットを計算する。"'
    pass